# Homework 6 _ JSON
# Santiago Ruiz Piedrahita

HOMEWORK

En un notebook de jupyter desarrolle los siguientes pasos

Descargue el JSON con la lista de paises del siguiente link:
https://datahub.io/core/country-list/r/data.json
y escoja aleatoriamente un país de Europa 
(Ejemplo que es abajo usa Colombia pero con pais de europa es mas facil)

Use los diferentes API endpoints de inspire-hep 
https://inspirehep.net/ 

Para extraer la lista de investigadores de una institución de ese país  en esa base de datos. 


Para ello:

A) Use el API de institutions para extraer la lista de instituciones del país, por ejemplo: colombia (mejor si es Europa, por completo de datos )
https://inspirehep.net/api/institutions?q=colombia


Para la primera institución con

number_of_papers > 0


Obtenga el valor

legacy_ICN:


Por ejemplo: 
Colombia, U. Natl.


Si ninguna institución satisface la condición number_of_papers > 0, escoja de nuevo otro país aleatorio y repita el proceso



In [1]:
# librerias
import json
import requests
import numpy as np
import pandas as pd

In [2]:
# leemos los datos
data = open("data_json.json")

In [3]:
# eligiendo un pais aleatoriamente
pais = "Estonia"

In [4]:
# veamos las instituciones
URL = "https://inspirehep.net/api/institutions?q={}".format(pais)
institucion = requests.get(URL)

legacy_ICN = []

for i in institucion.json().get('hits').get('hits'):

    legacy = i.get('metadata').get('legacy_ICN')
    papers = i.get('metadata').get('number_of_papers')

    if papers>0:
        legacy_ICN.append(legacy)

legacy_ICN

['Unlisted, EE',
 'Tartu, Inst. Phys.',
 'Tartu, Inst. Astrophys.',
 'Tartu Observ.',
 'Estonian U.',
 'Tartu State U.',
 'Tallinn Polytechnic Inst.',
 'Comp. Sci. Coll., Tallinn',
 'Tallinn Pedagogical U.',
 'Estonian Agricultural U.']

B) Con el API de literatura obtenga el JSON con los artículos de menos de 10 autores usando el "legacy_ICN" de la siguiente manera

https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+Colombia,+U.+Natl.+and+ac+1->+10


aff: usa el valor de legacy_ICN
and: es un operador lógico
ac: establece los autores entre 1 y 10



In [11]:
link = legacy_ICN[0].replace(' ','+')
lit ="https://inspirehep.net/api/literature?sort=mostrecent&page=1&q=aff+{}+and+ac+1->+10".format(link)
lit = requests.get(lit)


art = []
for i in lit.json().get('hits').get('hits'):
  art.append (i.get('metadata')) 

len(art)

7

C) Para al menos un artículo de esa institución, extraiga el URL del perfil de cada autor de esa institución que se encuentra dentro del campo "authors" en "record" y luego en "$ref". Por ejemplo

https://inspirehep.net/api/authors/1010271




In [12]:
# extrayendo los autores
autores = []
for i in art[0].get('authors'):
  autores.append(i.get('record').get('$ref')) 

autores

['https://inspirehep.net/api/authors/1259434',
 'https://inspirehep.net/api/authors/1334455',
 'https://inspirehep.net/api/authors/1274396']

D) Con cada uno de los datos del resultado del API para cada perfil construya una tabla con los siguientes columnas (puede que alguno de los datos no esté disponible): 
Nombre Completo
Correo electrónico
posición más reciente (la primera  que aparece en la lista "positions" del JSON) con su correspondiente:
rango 
institución 
fecha de inicio 
fecha de finalización

In [18]:
# para guardar los datos
nombre = []
email = []
posicion = []
institucion = []
F_inicio = []
F_finalizacion = []

In [19]:
for i in autores:

  info = requests.get(i).json().get('metadata')

  name = info.get('name').get('value')
  nombre.append(name)

  email_a = info.get('email_addresses')[0].get('value')
  email.append(email_a)

  rank = info.get('positions')[0].get('rank')
  posicion.append(rank)

  institution = info.get('positions')[0].get('institution') #institution where they  had their last position
  institucion.append(institution)

  start = info.get('positions')[0].get('start_date')
  F_inicio.append(start)

  end = info.get('positions')[0].get('end_date')
  F_finalizacion.append(end)  

In [20]:
# guardamos los datos
informacion = {"Nombre":nombre,"Email":email,"Posicion":posicion,"Institucion":institucion,"F_inicio":F_inicio,"F_finalizacion":F_finalizacion}
pd.DataFrame(informacion)

,Nombre,Email,Posicion,Institucion,F_inicio,F_finalizacion
0,"Lewicki, Marek",marek.lewicki@fuw.edu.pl,None,Warsaw U.,2020,None
1,"Vaskonen, Ville",vvaskonen@ifae.es,POSTDOC,Padua U.,2022,None
2,"Veermäe, Hardi",hardi.veermae@cern.ch,None,"NICPB, Tallinn",None,None
